# README
## Options:
### Pretrained model:
- word2vec-ruscorpora-300	
- glove-twitter-200

### Tokenization
- How to deal with OOV?
    - Mention -> '@'
    - Hashtag -> '#'
    - Link -> '&'

# Collective Function

In [ ]:
import pandas as pd
import numpy as np
# nltk.download('punkt') # nltk.download('averaged_perceptron_tagger')
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer
import gensim
import copy

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
model = api.load('glove-twitter-200')

In [212]:
def vectorize(line):
    words = []
    for word in line:  # line - iterable, for example list of tokens
        try:
            w2v_idx = w2v_indices[word]
        except KeyError:  # if you does not have a vector for this word in your w2v model, continue
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            # words.append(np.zeros((200,)))
            words.append(None)
        if len(line) > len(words):
            continue
    return np.asarray(words)

# insert only text data
def get_W2V_AVG(raw_data):
    tweet_tokenizer = TweetTokenizer()
    tweet_tokens = []
    stop_words = set(stopwords.words('english'))

    for sent in raw_data.text:
        sent = re.sub('', '', sent.lower())
        sent = re.sub(r'([^\s\w@#]|_)+', '', sent)
        sent = re.sub(r"http\S+", "&", sent)
        sent = re.sub(r"@\S+", '@', sent)
        sent = re.sub(r"#\S+", '#', sent)
        sent = [tweet_tokenizer.tokenize(sent)]
        sent = [token for token in sent[0] if not token in stop_words]
        tweet_tokens.append([sent])
    df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])

    w2v_object = model.wv
    # here you load vectors for each word in your model
    w2v_vectors = w2v_object.vectors
    # here you load indices - with whom you can find an index of the particular word in your model
    w2v_indices = {
        word: w2v_object.vocab[word].index for word in w2v_object.vocab}

    df_tokens['token_vec_avg'] = copy.deepcopy(df_tokens['token'])

    # for index, sent in enumerate(df_tokens['token_vec_avg']):
    for index, sent in enumerate(df_tokens['token_vec_avg']):
        df_tokens['token_vec_avg'][index] = vectorize(sent).mean(axis=0)

    df_temp = pd.DataFrame(df_tokens['token_vec_avg'].values.tolist()).add_prefix('W2V_avg')  # .join(df)
    df_tokens = df_tokens.join(df_temp).drop('token_vec_avg', axis=1)
    return df_tokens

In [216]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")
df_tokens = get_W2V_AVG(raw_data)
df_tokens.head()

raw_rumorhasit = pd.read_csv("./data/_RHS_text.csv")
rhi_tokens = get_W2V_AVG(raw_rumorhasit)
# rhi_tokens.head()

'''Word2Vec Averaged Vector Feature set'''
raw_rumorhasit.to_csv('./data/_PHEME_text_AVGw2v.csv', index = False)
rhi_tokens.to_csv('./data/_RHS_text_AVGw2v.csv', index = False)

In [198]:
# for i in df_tokens['token']:
#     if len(i) < 5 :
#         print(i)   

# Data Import

In [199]:
import pandas as pd
import numpy as np

import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stpdfa
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# import matplotlib as plot

In [3]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 400)

In [200]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")

# Preprocessing

# Tokenization

In [80]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

stop_words = set(stopwords.words('english'))
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []
stop_words = set(stopwords.words('english'))

for sent in raw_data.text:
    sent = re.sub(r"http\S+", "", sent)
    sent = re.sub(r'([^\s\w]|_)+', '', sent)
    sent = re.sub('', '', sent.lower())
    sent = [tweet_tokenizer.tokenize(sent)]
    sent = [token for token in sent[0] if not token in stop_words]
    # tweet_tokens.append(sent)
    tweet_tokens.append([sent])
# print(tweet_tokens)
df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])
# [[['BREAKING', 'Armed', 'man', 'takes', 'hostage', 'in', 'kosher', 'grocery', 'east', 'of', 'Paris']],
df_tokens.head(3)

,token
0,"[breaking, armed, man, takes, hostage, kosher, grocery, east, paris]"
1,"[charliehebdo, killers, dead, confirmed, gendarmerie]"
2,"[top, french, cartoonists, charb, cabu, wolinski, tignous, confirmed, among, dead, paris, charliehebdo, attack, editor, critically, wounded]"


# Word2Vec

## Downloading Pretrained model for Gensim

## Fetching pretrained Model and Convert the Raw Text

In [8]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
# corpus = api.load('text8')
# wv = api.load('word2vec-google-news-300')
# fasttext-wiki-news-subwords-300'
#  'glove-twitter-200',
model = api.load('glove-twitter-200')

In [12]:
# Handling the OOV
# for doc in df_tokens:
#     words = filter(lambda x: x in model.vocab, doc)
#     print(words)
# for word, i in t.word_index.items():
#     try:
#         embedding_vector = model.wv[word] except: print(word, 'not found')


In [13]:
w2v_object = model.wv
w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [72]:
def vectorize(line):
    # print(line)
    # print(len(np.asarray(line)))
    words = []
    for word in line:  # line - iterable, for example list of tokens
        try:
            w2v_idx = w2v_indices[word]
        except KeyError:  # if you does not have a vector for this word in your w2v model, continue
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            # words.append(np.zeros((200,)))
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)
    # print(words)
    # print(len(np.asarray(words)))
    # return words


def get_vectorOOV(s):
    try:
        return np.array(model.wv.get_vector(s))
    except KeyError:
        return np.zeros((300,))


In [170]:
# print("Tweet: ", raw_data['text'][1])
# print("Tweet Token: ", df_tokens['token'][10])
# print("Indice of '{}': {}".format(df_tokens['token'][10][0], w2v_indices[df_tokens['token'][10][0]]))
# print("Indice of '{}': {}".format(df_tokens['token'][10][0], w2v_vectors[w2v_indices[df_tokens['token'][10][0]]]))
# print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_indices[raw_data['text_token'][1][1]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_vectors[w2v_indices[raw_data['text_token'][1][1]]]))
# print("\nVector of the first headline can be avaraged:\n", vectorize(df_tokens['token'][10]).mean())
# w2v_vectors[w2v_indices[df_tokens['token'][10][0]]].shape
# print(w2v_indices['@'])
# len(vectorize(df_tokens['token'][10]))
# vectorize(df_tokens['token'][10]).mean(axis=0).shape

304


In [66]:
df_tokens#.token[10]

,token
0,"[breaking, armed, man, takes, hostage, kosher, grocery, east, paris]"
1,"[charliehebdo, killers, dead, confirmed, gendarmerie]"
2,"[top, french, cartoonists, charb, cabu, wolinski, tignous, confirmed, among, dead, paris, charliehebdo, attack, editor, critically, wounded]"
3,"[police, surrounded, area, charliehebdo, attack, suspects, believed]"
4,"[photo, armed, gunmen, face, police, officers, near, charliehebdo, hq, paris]"
...,...
5797,"[ill, ride, australia, unites, sydneysiege]"
5798,"[canadas, thoughts, prayers, australian, friends, martinplace, sydneysiege]"
5799,"[every, nonmuslim, world, must, watch, video, amp, show, every, nonmuslim, sydneysiege]"
5800,"[suspect, sydney, cafe, siege, identified, man, haron, monis, iranian, granted, asylum, australia]"


In [81]:
import copy
df_tokens['token_vec_avg'] = copy.deepcopy(df_tokens['token'])

# for index, sent in enumerate(df_tokens['token_vec_avg']):
for index, sent in enumerate(df_tokens['token_vec_avg']):
    df_tokens['token_vec_avg'][index] = vectorize(sent).mean(axis=0)

df_temp = pd.DataFrame(df_tokens['token_vec_avg'].values.tolist()).add_prefix(
    'W2V_avg')  # .join(df)
df_tokens = df_tokens.join(df_temp).drop('token_vec_avg', axis=1)
df_tokens.head()

,token,W2V_avg0,W2V_avg1,W2V_avg2,W2V_avg3,W2V_avg4,W2V_avg5,W2V_avg6,W2V_avg7,W2V_avg8,W2V_avg9,W2V_avg10,W2V_avg11,W2V_avg12,W2V_avg13,W2V_avg14,W2V_avg15,W2V_avg16,W2V_avg17,W2V_avg18,W2V_avg19,W2V_avg20,W2V_avg21,W2V_avg22,W2V_avg23,W2V_avg24,W2V_avg25,W2V_avg26,W2V_avg27,W2V_avg28,W2V_avg29,W2V_avg30,W2V_avg31,W2V_avg32,W2V_avg33,W2V_avg34,W2V_avg35,W2V_avg36,W2V_avg37,W2V_avg38,W2V_avg39,W2V_avg40,W2V_avg41,W2V_avg42,W2V_avg43,W2V_avg44,W2V_avg45,W2V_avg46,W2V_avg47,W2V_avg48,W2V_avg49,W2V_avg50,W2V_avg51,W2V_avg52,W2V_avg53,W2V_avg54,W2V_avg55,W2V_avg56,W2V_avg57,W2V_avg58,W2V_avg59,W2V_avg60,W2V_avg61,W2V_avg62,W2V_avg63,W2V_avg64,W2V_avg65,W2V_avg66,W2V_avg67,W2V_avg68,W2V_avg69,W2V_avg70,W2V_avg71,W2V_avg72,W2V_avg73,W2V_avg74,W2V_avg75,W2V_avg76,W2V_avg77,W2V_avg78,W2V_avg79,W2V_avg80,W2V_avg81,W2V_avg82,W2V_avg83,W2V_avg84,W2V_avg85,W2V_avg86,W2V_avg87,W2V_avg88,W2V_avg89,W2V_avg90,W2V_avg91,W2V_avg92,W2V_avg93,W2V_avg94,W2V_avg95,W2V_avg96,W2V_avg97,W2V_avg98,W2V_avg99,W2V_avg100,W2V_avg101,W2V_avg102,W2V_avg103,W2V_avg104,W2V_avg105,W2V_avg106,W2V_avg107,W2V_avg108,W2V_avg109,W2V_avg110,W2V_avg111,W2V_avg112,W2V_avg113,W2V_avg114,W2V_avg115,W2V_avg116,W2V_avg117,W2V_avg118,W2V_avg119,W2V_avg120,W2V_avg121,W2V_avg122,W2V_avg123,W2V_avg124,W2V_avg125,W2V_avg126,W2V_avg127,W2V_avg128,W2V_avg129,W2V_avg130,W2V_avg131,W2V_avg132,W2V_avg133,W2V_avg134,W2V_avg135,W2V_avg136,W2V_avg137,W2V_avg138,W2V_avg139,W2V_avg140,W2V_avg141,W2V_avg142,W2V_avg143,W2V_avg144,W2V_avg145,W2V_avg146,W2V_avg147,W2V_avg148,W2V_avg149,W2V_avg150,W2V_avg151,W2V_avg152,W2V_avg153,W2V_avg154,W2V_avg155,W2V_avg156,W2V_avg157,W2V_avg158,W2V_avg159,W2V_avg160,W2V_avg161,W2V_avg162,W2V_avg163,W2V_avg164,W2V_avg165,W2V_avg166,W2V_avg167,W2V_avg168,W2V_avg169,W2V_avg170,W2V_avg171,W2V_avg172,W2V_avg173,W2V_avg174,W2V_avg175,W2V_avg176,W2V_avg177,W2V_avg178,W2V_avg179,W2V_avg180,W2V_avg181,W2V_avg182,W2V_avg183,W2V_avg184,W2V_avg185,W2V_avg186,W2V_avg187,W2V_avg188,W2V_avg189,W2V_avg190,W2V_avg191,W2V_avg192,W2V_avg193,W2V_avg194,W2V_avg195,W2V_avg196,W2V_avg197,W2V_avg198,W2V_avg199
0,"[breaking, armed, man, takes, hostage, kosher, grocery, east, paris]",-0.214452,0.022334,0.199403,0.091531,-0.106364,-0.135615,0.224626,-0.100319,0.156632,-0.080552,-0.187510,0.085141,-0.692078,0.294819,0.190482,-0.000107,0.130387,-0.086283,0.000389,-0.256427,0.116084,-0.121319,-0.104006,-0.152930,0.023462,0.883121,0.178624,-0.068786,-0.162735,-0.025268,-0.081778,-0.072879,0.013534,-0.211057,0.161137,0.328094,-0.083515,0.074648,0.294723,-0.128192,0.191932,-0.004869,-0.009566,-0.111364,0.023465,0.179786,0.075096,-0.021256,-0.073478,0.147782,-0.470354,0.122131,-0.054976,0.097603,-0.216612,0.179325,-0.135947,0.018460,-0.087136,0.080001,0.104294,0.091111,0.128683,-0.013114,0.117404,-0.043849,0.204457,0.150360,-0.056318,-0.097586,-0.176027,-0.034084,0.131073,0.023279,0.092683,0.058691,-0.028683,-0.015163,0.025195,0.051936,0.374640,-0.091818,0.210236,0.206264,-0.073652,-0.212403,-0.131285,0.245320,-0.031517,-0.134470,-0.019555,-0.028470,0.130524,0.108279,-0.115213,0.134851,0.071441,0.061887,0.032899,-0.012026,0.299926,-0.140451,-0.122578,-0.307877,0.121167,-0.329101,0.065360,0.072052,0.156233,-0.119539,0.102322,-0.157276,0.073717,-0.007854,-0.000111,-0.240686,-0.007508,0.180650,-0.256752,0.362714,0.035268,0.046164,-0.035225,0.083667,-0.241622,-0.167557,0.108498,0.164607,0.431350,0.315716,-0.253180,0.140676,-0.126034,-0.474589,-0.084879,-0.358264,0.066130,0.110956,0.159029,-0.050851,-0.025120,0.258623,-0.202793,-0.207713,-0.221215,-0.004675,0.036970,-0.061748,0.001078,0.154303,0.091478,-0.095547,-2.876716,-0.078268,0.229267,-0.062095,-0.024084,0.122586,0.170145,0.006689,-0.100560,0.164260,-0.060955,0.046130,0.185162,-0.142742,-0.143107,-0.036575,0.335150,-0.313509,-0.235394,0.058141,-0.062775,0.226615,0.019887,-0.620922,0.119913,-0.163845,-0.269272,0.167251,-0.132802,-0.105065,-0.206132,0.153061,0.089953,0.093279,0.211769,-0.351812,0.060335,0.287951,0.113106,-0.158723,0.0463

# Exporting data

In [ ]:
'''Word2Vec Averaged Vector Feature set'''
df_train_avg.to_csv('./data/train_avg.csv', index = False)
df_test_avg.to_csv('./data/test_avg.csv', index = False)